## Background exposure calculation for each volume of the detector

In [1]:
import pandas as pd
import numpy as np

We read and compute the efficiences for each volume and isotope (214Bi, 208Tl, etc)

In [2]:
#read file
efficiencies = pd.read_csv('efficiencies_2.4_2.5_MeV.csv')
#compute efficiencies
efficiencies['eff'] = efficiencies.nsaved / efficiencies.nsim
efficiencies['eff_Qbb'] = efficiencies.nenergy / efficiencies.nsim

efficiencies.drop(['nsim', 'nsaved', 'npass', 'nenergy'], axis = 1, inplace=True)

We read the activities now and set up the DF to merge with the efficiencies DF.

In [3]:
#read and rename activities file
activities = pd.read_csv('activities_NEXT100.csv', sep=';', decimal = ',')
activities = activities.rename(columns = {'NEXUS VOLUME':'G4Volume', 'Bi-214':'214Bi', 'Co-60':'60Co', 'K-40':'40K', 'Tl-208':'208Tl'})

Bi_act = activities[['G4Volume', '214Bi']].copy()
Tl_act = activities[['G4Volume', '208Tl']].copy()
Bi_act['Isotope'] = '214Bi'
Tl_act['Isotope'] = '208Tl'

act = Bi_act.rename(columns={'214Bi':'act'}).append(Tl_act.rename(columns={'208Tl':'act'}))

#merge activities and efficiencies 
eff_and_act = efficiencies.merge(act, on = ['G4Volume', 'Isotope'], how = 'outer')


We add a column with the counts/year we want to have (using the ones in the 2.4-2.5MeV region from Gonzalo's thesis)

In [4]:
#total_final_counts stands for the number of counts we want to have from one isotope
eff_and_act['total_final_counts'] = eff_and_act['Isotope'].apply(lambda x: {'208Tl':1800, 
                                                                            '214Bi':296, 
                                                                            '0nubb':0, 
                                                                            'muons':900, 
                                                                            '137Xe':33}[x])

In [5]:
eff_and_act.head(1)

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.0,296


At this point we have a df with the columns:
* **Isotope**: the name of the isotope
* **G4Volume**: the name of the volume
* **eff**: nexus efficiency for each isotope and volume
* **eff_Qbb**: energy efficiency for each isotope and volume
* **act**: activity (mBq) for each isotope and volume
* **total_final_counts**: number of counts we want to have for a specific isotope, which are going to be distributed in each volume

We now will normalize for each isotope the efficiency x activity to compute the required counts per volume. For that I create the following function:

In [6]:

def distribute_final_counts(eff_and_act, eff = 'eff'):
    '''
    Function that distributes the desired counts per isotope into the different volumes attending to their
    efficiency and activity. It computes a normalized factor of efficiency times activity per volume that
    we can use to distribute the event counts in each volume. Finally the counts are rounded up.

    Variables:
        eff_and_act: DF
            DataFrame that contains the information per isotope and volume of their efficiencies and activities, and
            also the total desired counts for an isotope
        eff: STR
            Selects to compute the regular efficiency (nexus efficiency for each volume) or the energy efficiency (events 
            in the desired energy)
    '''
    #multiply the efficiency and the activity
    eff_and_act['eff_x_act'] = eff_and_act[eff] * eff_and_act['act']

    #sum all those factors to normalize
    sum_eff_x_act = pd.DataFrame(eff_and_act.groupby('Isotope')['eff_x_act'].sum()).reset_index().rename({'eff_x_act':'sum_eff_x_act'}, axis = 1)
    #merge and divide
    eff_and_act = eff_and_act.merge(sum_eff_x_act, on = 'Isotope', how = 'outer')
    eff_and_act['norm_eff_x_act'] = eff_and_act['eff_x_act'] / eff_and_act['sum_eff_x_act']

    #decide which efficiency to use
    if eff == 'eff_Qbb': final_name = 'final_counts_Qbb'
    else: final_name = 'final_counts'

    #distribute the counts according to the computed factor, round up the counts and clean the DF
    eff_and_act[final_name] = (eff_and_act['total_final_counts'] * eff_and_act['norm_eff_x_act']).apply(np.ceil)
    eff_and_act.drop(['eff_x_act', 'sum_eff_x_act', 'norm_eff_x_act'], axis = 1, inplace = True)
    return eff_and_act

We now apply the function, two times to do it with both efficiencies

In [7]:
eff_and_act = distribute_final_counts(eff_and_act)
eff_and_act = distribute_final_counts(eff_and_act, eff='eff_Qbb')

In [8]:
eff_and_act.head(1)

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts,final_counts,final_counts_Qbb
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.0,296,1.0,1.0


Now we have two more columns:
* **final_counts**: the distributed counts per volume for nexus efficiency
* **final_counts_Qbb**: the distributed counts per volume for energy efficiency

Finally we select a factor to increase the statistics (in this case x100) and compute the required simulation counts and the exposure.

In [9]:
factor = 100

#for activities in mBq to year^-1
year = (3600 * 24 * 365) / 1000

eff_and_act['needed_counts'] = (eff_and_act.final_counts / eff_and_act.eff) * factor
eff_and_act['needed_counts_Qbb'] = (eff_and_act.final_counts_Qbb / eff_and_act.eff_Qbb) * factor
eff_and_act['exposure'] = eff_and_act.needed_counts / (eff_and_act.act * year)
eff_and_act['exposure_Qbb'] = eff_and_act.needed_counts_Qbb / (eff_and_act.act * year)


In [10]:
eff_and_act.head(1)

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts,final_counts,final_counts_Qbb,needed_counts,needed_counts_Qbb,exposure,exposure_Qbb
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.0,296,1.0,1.0,999600250.0,3.332001e+09,204.497523,681.658408


So finally we end with a DF with:
* Isotope
* G4Volume
* eff
* eff_Qbb
* act
* total_final_counts
* final_counts
* final_counts_Qbb
* **needed_counts**: required counts for nexus simulation to obtain the desired quantity using nexus efficiency
* **needed_counts_Qbb**: required counts for nexus simulation to obtain the desired quantity using energy efficiency
* **exposure**: exposure to obtain the desired counts in nexus
* **exposure_Qbb**: exposure to obtain the desired counts in nexus and in the energy window (2.4-2.5MeV)

In [11]:
eff_and_act

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts,final_counts,final_counts_Qbb,needed_counts,needed_counts_Qbb,exposure,exposure_Qbb
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.000,296,1.0,1.0,9.996002e+08,3.332001e+09,204.497523,681.658408
1,214Bi,EP_COPPER_PLATE,3.802124e-06,1.225071e-06,313.000,296,5.0,6.0,1.315054e+08,4.897674e+08,13.322717,49.617967
2,214Bi,GATE_RING,3.953236e-05,1.115436e-05,4.760,296,1.0,1.0,2.529573e+06,8.965106e+06,16.851313,59.723034
3,214Bi,HDPE_TUBE,4.443023e-05,1.370990e-05,3.070,296,1.0,1.0,2.250720e+06,7.294000e+06,23.247512,75.339160
4,214Bi,VESSEL,1.520359e-07,5.307350e-08,3420.000,296,2.0,3.0,1.315479e+09,5.652538e+09,12.196948,52.409595
5,214Bi,PEDESTAL,2.500000e-08,6.250000e-09,358.000,296,1.0,1.0,4.000000e+09,1.600000e+10,354.299352,1417.197407
6,214Bi,SHIELDING_LEAD,2.936508e-08,8.888889e-09,5320.000,296,1.0,1.0,3.405405e+09,1.125000e+10,20.297875,67.055481
7,214Bi,ANODE_RING,4.014916e-05,9.302854e-06,4.760,296,1.0,1.0,2.490712e+06,1.074939e+07,16.592430,71.609434
8,214Bi,SIPM_BOARD,8.291265e-05,1.696023e-05,21.800,296,7.0,5.0,8.442620e+06,2.948073e+07,12.280446,42.882010
9,214Bi,OPTICAL_PAD,2.528092e-05,8.263023e-06,7.920,296,1.0,1.0,3.955553e+06,1.210211e+07,15.837091,48.453957
